In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Date
0,World,2827435,104266.0,196956.0,6039.0,780085.0,1850394,58367.0,363.00,25.3,0.0,0.0,2020-04-24 19:25:07.417407
1,USA,923474,37032.0,52085.0,1851.0,93369.0,778020,14954.0,2790.00,157.0,5012706.0,15144.0,2020-04-24 19:25:07.417407
2,Spain,219764,6740.0,22524.0,367.0,92355.0,104885,7705.0,4700.00,482.0,930230.0,19896.0,2020-04-24 19:25:07.417407
3,Italy,192994,3021.0,25969.0,420.0,60498.0,106527,2173.0,3192.00,430.0,1642356.0,27164.0,2020-04-24 19:25:07.417407
4,France,159828,1645.0,22245.0,389.0,43493.0,94090,4870.0,2449.00,341.0,463662.0,7103.0,2020-04-24 19:25:07.417407
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Sao Tome and Principe,4,0.0,0.0,0.0,0.0,4,0.0,18.00,0.0,19.0,87.0,2020-04-24 19:25:07.417407
209,Anguilla,3,0.0,0.0,0.0,1.0,2,0.0,200.00,0.0,0.0,0.0,2020-04-24 19:25:07.417407
210,Saint Pierre Miquelon,1,0.0,0.0,0.0,0.0,1,0.0,173.00,0.0,0.0,0.0,2020-04-24 19:25:07.417407
211,Yemen,1,0.0,0.0,0.0,1.0,0,0.0,0.03,0.0,120.0,4.0,2020-04-24 19:25:07.417407


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,...,Tests/ 1M pop,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Spain,219764,6740.0,22524.0,367.0,92355.0,104885,7705.0,4700.0,482.0,...,19896.0,2020-04-24 19:25:07.417407,-3.647550,40.244487,114879.0,0.196067,0.102492,0.117975,0.012091,Last 8 columns are added by Roaya
2,Italy,192994,3021.0,25969.0,420.0,60498.0,106527,2173.0,3192.0,430.0,...,27164.0,2020-04-24 19:25:07.417407,12.070013,42.796626,86467.0,0.300334,0.134559,0.088797,0.011948,Last 8 columns are added by Roaya
3,France,159828,1645.0,22245.0,389.0,43493.0,94090,4870.0,2449.0,341.0,...,7103.0,2020-04-24 19:25:07.417407,2.618787,47.824905,65738.0,0.338389,0.139181,0.067510,0.009396,Last 8 columns are added by Roaya
0,USA,923474,37032.0,52085.0,1851.0,93369.0,778020,14954.0,2790.0,157.0,...,15144.0,2020-04-24 19:25:07.417407,-97.922211,39.381266,145454.0,0.358086,0.056401,0.149374,0.008425,Last 8 columns are added by Roaya
177,China,82804,6.0,4632.0,0.0,77257.0,915,57.0,58.0,3.0,...,0.0,2020-04-24 19:25:07.417407,103.819074,36.561765,81889.0,0.056564,0.055939,0.084096,0.004704,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,Madagascar,122,1.0,0.0,0.0,61.0,61,1.0,4.0,0.0,...,85.0,2020-04-24 19:25:07.417407,46.704737,-19.371896,61.0,0.000000,0.000000,0.000063,0.000000,Last 8 columns are added by Roaya
125,Cambodia,122,0.0,0.0,0.0,110.0,12,1.0,7.0,0.0,...,345.0,2020-04-24 19:25:07.417407,104.906943,12.720048,110.0,0.000000,0.000000,0.000113,0.000000,Last 8 columns are added by Roaya
119,Rwanda,176,22.0,0.0,0.0,87.0,89,0.0,14.0,0.0,...,537.0,2020-04-24 19:25:07.417407,29.919885,-1.990338,87.0,0.000000,0.000000,0.000089,0.000000,Last 8 columns are added by Roaya
114,Vietnam,270,2.0,0.0,0.0,225.0,45,8.0,3.0,0.0,...,2119.0,2020-04-24 19:25:07.417407,106.299147,16.646017,225.0,0.000000,0.000000,0.000231,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"USA","TotalCases":923474,"NewCases":37032.0,"TotalDeaths":52085.0,"NewDeaths":1851.0,"TotalRecovered":93369.0,"ActiveCases":778020,"CriticalCases":14954.0,"Tot\\u00a0Cases\\/1M pop":2790.0,"Deaths\\/1M pop":157.0,"TotalTests":5012706.0,"Tests\\/ 1M pop":15144.0,"Date":1587756307417,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":145454.0,"ClosedDeathRate":0.3580857178,"CurrentDeathRate":0.0564011548,"Weight":0.1493741759,"DeathRateXWeight":0.008424876,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spain","TotalCases":219764,"NewCases":6740.0,"TotalDeaths":22524.0,"NewDeaths":367.0,"TotalRecovered":92355.0,"ActiveCases":104885,"CriticalCases":7705.0,"Tot\\u00a0Cases\\/1M pop":4700.0,"Deaths\\/1M pop":482.0,"TotalTests":930230.0,"Tests\\/ 1M pop":19896.0,"Date":1587756307417,"Longitude":-3.64755047,"Latitude":40.24448698,"ClosedCases":114879.0,"ClosedDeathRate":0.1960671663,"CurrentDeathRate":0.1024917639,"Weight":0.1179751396,"DeathRateXWeight":0.012091